In [1]:
# 라이브러리 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options


In [2]:
#크롤링 시작과 끝 날짜(달 별로 크롤링)
#                                                        각 달의 첫날
first_days=pd.date_range('2021/01/01', '2021/01/31', freq='MS')
#                                                        각 달의 마지막 날
last_days=pd.date_range('2021/01/01', '2021/01/31', freq='M')

#검색 키워드 추가
keyword_list=['이수역맛집', '점심']

In [3]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드


chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치


In [4]:
# 블로그 주소 수집

blog_url_list = []

# 네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 990 #
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭
    
    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()
            
            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30)-1 # 한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                # 0부터 연재까지의 스크롤 좌표를 가져와 줘의 의미
                time.sleep(random.uniform(1, 1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                
                last_height = new_height
                
            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_='api_txt_lines total_tit')
            
            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) # 주소 수집
            
            print('{0} 키워드 {1}년 {2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url), '개 블로그 url 주소 수집')
            
            driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass

C:\Users\user\AppData\Local\Temp\ipykernel_11584\2598069413.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = chrome_path, chrome_options=chrome_options)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.107)


In [43]:
#수집 후 url list 중복 확인 810+990
real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

1724

In [44]:
#파일 저장(블로그 주소)
k = pd.DataFrame(blog_url_list)
#
k.to_csv('url_list_맛집.csv', encoding='utf-8-sig')

In [45]:
#
url=pd.read_csv('url_list_맛집.csv')
blog_url_list = list(url['0'])

In [46]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_5768\3172629000.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


10개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://mintmant.tistory.com/55
20개 블로그 크롤링 완료
30개 블로그 크롤링 완료
40개 블로그 크롤링 완료
50개 블로그 크롤링 완료
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://felkmt.tistory.com/4157
80개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://gqinside.tistory.com/412
90개 블로그 크롤링 완료
100개 블로그 크롤링 완료
110개 블로그 크롤링 완료
120개 블로그 크롤링 완료
130개 블로그 크롤링 완료
140개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://mintmant.tistory.com/60
150개 블로그 크롤링 완료
160개 블로그 크롤링 완료
170개 블로그 크롤링 완료
180개 블로그 크롤링 완료
190개 블로그 크롤링 완료
200개 블로그 크롤링 완료
210개 블로그 크롤링 완료
220개 블로그 크롤링 완료
230개 블로그 크롤링 완료
240개 블로그 크롤링 완료
250개 블로그 크롤링 완료
260개 블로그 크롤링 완료
270개 블로그 크롤링 완료
280개 블로그 크롤링 완료
290개 블로그 크롤링 완료
300개 블로그 크롤링 완료
310개 블로그 크롤링 완료
320개 블로그 크롤링 완료
330개 블로그 크롤링 완료
340개 블로그 크롤링 완료
350개 블로그 크롤링 완료
360개 블로그 크롤링 완료
370개 블로그 크롤링 완료
380개 블로그 크롤링 완료
390개 블로그 크롤링 완료
400개 블로그 크롤링 완료
410개 블로그 크롤링 완료
420개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://mforu.tistory.com/721
430개 블로그 크롤링 완료
440개 블로그 크롤링 완료
450개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://chandori11.tistory.c

In [ ]:
len(time_list)

In [ ]:
data={'time':time_list, 'review': review_list, 'comment':comment_list, 'like':like_list, 'url':url_list}
df=pd.DataFrame(data)

print('크롤링 소요 시간: ', time.time()-start)
print('총 {}개 블로그 크롤링 완료'.format(count))
df

In [35]:
#블로그 주소 수집
blog_url_list=[]
#네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)

    driver.get(url)
    time.sleep(3)

    #한번에 천개가 한계: 네이버 특성
    url_want=990
    
    #검색 옵션 클릭
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() 

    #날짜 지정
    for k in range(len(first_days)):
        try:
            #시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()


            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()

            #스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            #한페이지에 기본으로 30개 글이 나옴
            scroll = (url_want/30)-1

            #스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight')
                time.sleep(random.uniform(1,1.7))

                new_height=driver.execute_script('return document.body.scrollHeight')

                if new_height == last_height:
                    break
                last_height = new_height

            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_='api_txt_lines total_tit')

            #url 주소 수집
            for i in blog_url:
                #주소 수집
                blog_url_list.append(i['href'])

            print('{0} 키워드 {1}년 {2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url), '개 블로그 url 주소 수집')

            driver.execute_script('window.scrollTo(0,0)')

        except:
            pass



C:\Users\user\AppData\Local\Temp\ipykernel_5768\671444373.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
